In [1]:
%load_ext autoreload
%autoreload 2

import logging
from dotenv import load_dotenv
load_dotenv()

logging.basicConfig(level=logging.INFO)

logger = logging.getLogger(__name__)

In [2]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from projects.core.chargebacks_disputes_analysis.main import Service

svc = Service(use_cache=False)
date = (datetime.now() - relativedelta(months=6)).strftime("%Y-%m-%d")
svc.set_df(date)

INFO:snowflake.snowpark:AST state has not been set explicitly. Defaulting to ast_enabled = True.
INFO:src.environment:Target: DEV
INFO:src.environment:Developer: FDDIFERD
INFO:src.environment:Schema Name: DEV_FDDIFERD
INFO:projects.core.chargebacks_disputes_analysis.main:Querying data from snowflake
INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 3.18.0, Python Version: 3.10.18, Platform: macOS-26.1-arm64-arm-64bit
INFO:snowflake.connector.connection:Connecting to GLOBAL Snowflake domain
INFO:snowflake.snowpark.session:Snowpark Session information: 
"version" : 1.42.0,
"python.version" : 3.10.18,
"python.connector.version" : 3.18.0,
"python.connector.session.id" : 23301361779830,
"os.name" : Darwin

INFO:src.connection.session:Created or replaced database ML_LAYER_DB and schema DEV_FDDIFERD
INFO:src.connection.session:Connected to Snowflake for project CORE
INFO:src.connection.session:  Account: "vx70256-sw09563"
INFO:src.connection.session:  User: "ML_LAYER

,REF_TXN_ID,TXN_ID,PROCESSOR,TYPE,MERCHANT_ACCOUNT,CARD_NETWORK,BRAND_SLUG,SKU_TYPE_SLUG,CHANNEL_SLUG,TRAFFIC_SOURCE_SHORT_NAME,BIN,OFFER_TYPE,COUNT
0,TXN_1413425062,1.413425e+09,adyen,dispute,TruthFinderECOM,visa,TruthFinder,Background,Affiliate,TKMT,414720,standard,1
1,TXN_1396490977,1.396491e+09,adyen,fraud,TruthFinderECOM,mastercard,TruthFinder,Background,PaidSearch,GOOGSRCH,552433,standard,1
2,TXN_1441544489,1.441544e+09,adyen,dispute,TruthFinderECOM,mastercard,TruthFinder,Background,Affiliate,CV4T,510624,standard,1
3,TXN_1440614081,1.440614e+09,adyen,fraud,InstantCheckmateECOM,visa,InstantCheckmate,Background,Affiliate,FUNM,414720,standard,1
4,TXN_1434503134,1.434503e+09,adyen,dispute,TruthFinderECOM,mastercard,TruthFinder,ReversePhoneLookup,Organic,OG,525475,standard,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9106904,bh6fggz1,1.452061e+09,braintree,settlement,Truthfinder_instant_2,mastercard,TruthFinder,Background,Affiliate,CV4T,527517,standard,1
9106905,qdwcp357,1.452069e+09,braintree,settlement,Truthfinder_instant_2,None,OmniWatch,BasicMonthly,InternalReferral,TRTHFDR,None,discount,1
9106906,qb70m6sn,1.452076e+09,braintree,settlement,Truthfinder_instant_2,None,TruthFinder,Background,Organic,OG,521400,standard,1
9106907,fh72wejc,NaN,braintree,settlement,Truthfinder_instant_2,visa,None,None,None,None,None,None,1


In [14]:
from projects.core.chargebacks_disputes_analysis.main import Level

level = [Level.BIN]
df = svc.get_threshold_levels_df(level)


In [18]:
df['VALUE'][
    df['LEVEL'] == 'BIN'
].to_list()

['521729',
 '552315',
 '528546',
 '557729',
 '526219',
 '525475',
 '542418',
 '552433',
 '483313',
 '406032',
 '546616',
 '518941',
 '406042',
 '515678',
 '478200',
 '442755',
 '547415',
 '434769',
 '486796',
 '521267',
 '552342',
 '431196',
 '475055',
 '512107',
 '527515',
 '483312',
 '544317',
 '410040',
 '420767',
 '441103',
 '426690',
 '426938',
 '461046',
 '521876',
 '442756',
 '442742',
 '544347',
 '525363',
 '510805',
 '470793',
 '531260',
 '426684',
 '549806',
 '483316',
 '537993',
 '521333',
 '512230',
 '534348',
 '414720',
 '546316',
 '515676',
 '517805',
 '424631',
 '414718',
 '414740',
 '440066',
 '438854',
 '438857',
 '427082',
 '403784',
 '518725',
 '434257',
 '410039',
 '473702',
 '457431',
 '481582',
 '444796',
 '434256',
 '517800',
 '403766',
 '446542',
 '448233',
 '539689',
 '435546',
 '464018',
 '434258',
 '473703',
 '409530',
 '413037',
 '406095',
 '536081',
 '542432',
 '414734',
 '601100',
 '400022',
 '439466',
 '539107',
 '517546',
 '432739',
 '438877',
 '436618',

In [ ]:
from projects.core.connection import get_session

session = get_session()

In [ ]:
from projects.core.chargebacks_disputes_analysis.queries.adyen import queries
from pandas import DataFrame, concat

df = DataFrame()
for query in queries:
    df = concat([df, session.sql(query).to_pandas()])

df.head()

In [ ]:
# set max width and cols for pandas
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

from projects.core.chargebacks_disputes_analysis.main import (
    get_combination_threshold_df
)

combinations_df = get_combination_threshold_df(df)

In [ ]:
combinations_df['Percent'] = (combinations_df['DELTA_PERCENTAGE'] * 100).round(1).astype(str) + '%'

In [ ]:
combinations_df.sort_values(by='DELTA_PERCENTAGE', ascending=False, inplace=True)
combinations_df